# Find Timepoints(.gs)

Reads from ID and Date columns, updates Timepoint

*IMPORTANT*: The first time a participant is tested is considered as timepoint 1, the 2nd as timepoint 2... and so on. It does not matter which year they did the study or how far apart two timepoints are from each other. This is just to better identify how many times a participant has actually completed a specific study. 

In [ ]:
function updateTimepoints() {
  var sheet = SpreadsheetApp.getActiveSpreadsheet().getActiveSheet();
  var dataRange = sheet.getDataRange();
  var data = dataRange.getValues();

  var participantIDCol = data[0].indexOf("Participant_ID");
  var dateCol = data[0].indexOf("Date");
  var timepointCol = data[0].indexOf("Timepoint");

  if (participantIDCol < 0 || dateCol < 0 || timepointCol < 0) {
    SpreadsheetApp.getUi().alert('One of the required columns ("Participant_ID", "Date", "Timepoint") is missing.');
    return;
  }

  var timepointTracker = {};

  for (var i = 1; i < data.length; i++) {
    var participantID = data[i][participantIDCol];
    var dateString = data[i][dateCol] ? Utilities.formatDate(new Date(data[i][dateCol]), Session.getScriptTimeZone(), "MM/dd/yyyy") : '';

    if (!participantID || !dateString) continue;

    var key = participantID + '-' + dateString;

    if (!timepointTracker.hasOwnProperty(key)) {

      var currentMaxTimepoint = Object.keys(timepointTracker)
        .filter(k => k.startsWith(participantID + '-'))
        .reduce((max, k) => Math.max(max, timepointTracker[k]), 0);
      timepointTracker[key] = currentMaxTimepoint + 1;
    }

    if (data[i][timepointCol] === '') {
      sheet.getRange(i + 1, timepointCol + 1).setValue(timepointTracker[key]);
    }
  }

  SpreadsheetApp.getUi().alert('Timepoints have been updated successfully.');
}
